In [ ]:
import inspect
import typing
from typing import NewType, List
import struct
import json


uint64_t = NewType('uint64_t', int)
int64_t = NewType('int64_t', int)
int64 = NewType('int64', int)

symbol_type = NewType('int64_t', int)


class name(str):
    pass

class symbol:
    pass

class public_key:
    pass

class signature:
    pass

class asset:
    def __init__(amount : int64_t = 0, symbol: symbol_type = 0):
        self.amount = amount
        self.symbol = symbol

    @classmethod
    def unpack(cls, raw):
        a = asset()
        a.amount, a.symbol = struct.unpack('Q8s', raw)
        return a

name_map = {
    int: 'uint64_t',
    int64: 'int64',
    int64_t: 'int64_t',
    uint64_t: 'uint64_t',
    str: 'string',
    bytes: 'bytes',
    asset: 'asset',
    public_key: 'public_key',
    signature: 'signature'
}

def table(table_name):
    def wrap_class(cls):
        print(table_name, cls)
        return cls
    return wrap_class

    print(args)
    cls = args[0]
    table_name = args[1]
    print(cls, table_name)
    print(dir(cls))

#@table('accounts')
class account:
    pass

def define_table(cls, table_name, primary_key_type):
    table =  {
            "name": table_name,
            "type": cls.__name__,
            "index_type": "i64",
            "key_names": [],
            "key_types": []
        },
    abi['tables'].append(table)

def action(func):
    sig = inspect.signature(func)
    s = {
      "name": func.__name__,
      "base": "",
      "fields": [
      ]
    }
    abi['structs'].append(s)
    for item in sig.parameters.values():
        _arg_name = item.name
        if _arg_name == '_from':
            _arg_name = 'from'
        try:
            if type(item.annotation) is typing._GenericAlias:
                if item.annotation._name == 'List':
                    element_type = item.annotation.__args__[0]
                    _type_name = name_map[element_type] + "[]"
            else:
                _type_name = name_map[item.annotation]
#            print('++++type name:', _type_name)
        except KeyError:
            if hasattr(item.annotation, '__name__'):
                _type_name = item.annotation.__name__
            else:
                _type_name = 'unknow_type'

        s['fields'].append({"name": _arg_name, "type": _type_name})        
        assert not item.annotation is inspect.Parameter.empty, func.__name__ + ' parameter type not specified'

    a = {"name": func.__name__, "type": func.__name__,"ricardian_contract": ""}
    abi['actions'].append(a)


In [2]:

abi = '''
{
    "version": "eosio::abi/1.0",
    "types": [],
    "structs": [],
    "actions": [],
    "tables": []
}
'''
abi = json.loads(abi)

@action
def transfer(_from: name, to: name, amount: asset, memo: str):
    pass

@action
def test(a: List[str], b: public_key, c: signature):
    pass

define_table(account, 'accounts', int64)

#print(abi)
print(json.dumps(abi, default=lambda x: x, sort_keys=False, indent=4, separators=(',', ': ')))


{
    "version": "eosio::abi/1.0",
    "types": [],
    "structs": [
        {
            "name": "transfer",
            "base": "",
            "fields": [
                {
                    "name": "from",
                    "type": "name"
                },
                {
                    "name": "to",
                    "type": "name"
                },
                {
                    "name": "amount",
                    "type": "asset"
                },
                {
                    "name": "memo",
                    "type": "string"
                }
            ]
        },
        {
            "name": "test",
            "base": "",
            "fields": [
                {
                    "name": "a",
                    "type": "string[]"
                },
                {
                    "name": "b",
                    "type": "public_key"
                },
                {
                    "name": "c",
                    "type